In [1]:
import ee
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import geemap
import os
import time
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import geopandas as gpd
import pandas as pd
import numpy as np
from mpi4py import MPI

In [2]:
##########################################################################################
# Set Parameters
##########################################################################################

# area of interest params
# choose bounding area format ('STATE', 'COUNTRY', 'BBOX', 'HUC', 'SHP'):
ROI = 'SHP'

# if ROI = BBOX or SHP (path to .geojson or .shp, otherwise ''):
IN_PATH = '/mnt/poseidon/remotesensing/arctic/data/vectors/supplementary/tundra_alaska/tundra_alaska.shp'
# if ROI = STATE or COUNTRY (administrative boundaries, otherwise None):
COUNTRY = None
# if ROI = HUC, state abbreviation for HUC, if STATE, fulls state name:
STATE = None # 'AK' 
# if ROI = HUC (list of HUC6 units):
HUCLIST = None # must be list: ['190604', '190603', '190602']

##########################################################################################

# buffer around point to find median of intersecting pixel values
POINTBUFFER = 30 # meters

##########################################################################################

# output file
DIR_PATH = '/mnt/poseidon/remotesensing/arctic/data/rasters/S1GRD/training_test04'
try:
    if os.path.isdir(DIR_PATH) == False:
        os.mkdir(DIR_PATH)
except Exception as e:
    pass

##########################################################################################

# data Information
IDCOL = 'Site Code'
SCALE = 10
BANDS = ['VV', 'HH', 'VH', 'HV']
start_date = date(2019, 6, 1)# Y-M-D (2019, 1, 1)
end_date = date(2019, 8, 31) # Y-M-D minus 5 for even 'days' intervals (6 days for 2020)
TIMESTEP = None # 'months', 'days', or None
DAYS = '' # if TIMESTEP = days
MONTHS = '' # if TIMESTEP = 'months': years * 12

In [3]:
##########################################################################################
# Create ee_to_df function for exporting
##########################################################################################

def ee_to_df(ee_object, col_names, sort_columns=False):
    if isinstance(ee_object, ee.Feature):
        ee_object = ee.FeatureCollection([ee_object])

    if not isinstance(ee_object, ee.FeatureCollection):
        raise TypeError("ee_object must be an ee.FeatureCollection")

    try:
        property_names = ee_object.first().propertyNames().sort().getInfo()
        #data = ee_object.map(lambda f: ee.Feature(None, f.toDictionary(property_names)))
        data = ee_object
        data = [x["properties"] for x in data.getInfo()["features"]]
        df = pd.DataFrame(data)

        if col_names is None:
            col_names = property_names
            col_names.remove("system:index")
        elif not isinstance(col_names, list):
            raise TypeError("col_names must be a list")

        df = df[col_names]

        if sort_columns:
            df = df.reindex(sorted(df.columns), axis=1)

        return df
    
    except Exception as e:
        raise Exception(e)

In [4]:
##########################################################################################
# Set Date Ranges
##########################################################################################
# days

if TIMESTEP == 'days':

    def create_list_of_dates(start_date, end_date):
        dates = []
        delta = end_date - start_date   # returns timedelta

        for i in range(delta.days + 1):
            day = start_date + timedelta(days=i)
            dates.append(day)
        return dates

    def create_time_intervals(dates_list, Interval):
        time_df = pd.DataFrame({'Date': dates_list}).astype('datetime64[ns]')
        interval = timedelta(Interval)
        grouped_cr = time_df.groupby(pd.Grouper(key='Date', freq=interval))
        date_ranges = []
        for i in grouped_cr:
            date_ranges.append(((str(i[1].min()[0]).split(' ')[0]), 
                                (str(i[1].max()[0]).split(' ')[0])))
        return date_ranges

    date_ranges = create_time_intervals(create_list_of_dates(start_date, 
                                                             end_date), 
                                        DAYS)
    print(date_ranges)

##########################################################################################
# months

elif TIMESTEP == 'months':

    def create_list_of_dates(start_date, end_date):

        dates = []
        end_date = end_date - relativedelta(months=MONTHS-1)
        for i in range(MONTHS):
            delta = relativedelta(months=i)
            month_start = start_date + delta
            month_end = end_date + delta
            dates.append((month_start.strftime('%Y-%m-%d'), 
                          month_end.strftime('%Y-%m-%d')))
        return dates

    date_ranges = create_list_of_dates(start_date, end_date)
    print(date_ranges)

##########################################################################################
# no step

elif TIMESTEP == None:

    date_ranges = [(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))]

else:

    print("Invalid TIMESTEP selection. Use 'days', 'months', or None.")

In [5]:
rank = 1
size = 4

In [6]:
##########################################################################################
# Set Ranks by Number of bands
##########################################################################################

# create band directories
allbands = np.array(BANDS)
allbands = np.array_split(allbands, size) # split array into x pieces
for r in range(len(allbands)):
    if r == rank:
        CURRENTBANDS = BANDS[r] # select object from list (could be str or lst)
        PATH = f'{DIR_PATH}/{CURRENTBANDS}'
    else:
        pass

print('RANK:', rank, CURRENTBANDS, flush = True)

# create timestamp directories within each huc (rank)
if os.path.isdir(PATH):
    pass
else:
    os.mkdir(PATH)

RANK: 1 HH


In [7]:
##########################################################################################
# Set GEE Vector Bounds
##########################################################################################

# Import admin data and select country to create grid around
if ROI == 'STATE':
    grid_location_ee = (ee.FeatureCollection("FAO/GAUL/2015/level1")
                        .filterMetadata('ADM0_NAME', 'equals', COUNTRY)
                        .filterMetadata('ADM1_NAME', 'equals', STATE))

elif ROI == 'COUNTRY':
    grid_location_ee = (ee.FeatureCollection("FAO/GAUL/2015/level1")
                        .filterMetadata('ADM0_NAME', 'equals', COUNTRY))
	
elif ROI == 'BBOX':
	grid_location_ee = geemap.geojson_to_ee(IN_PATH)
    
elif ROI == 'HUC':
    grid_location_ee = (ee.FeatureCollection("USGS/WBD/2017/HUC06")
                        .filter(ee.Filter.inList('huc6', HUCLIST)))
    
elif ROI == 'SHP':
    geodataframe = gpd.read_file(IN_PATH)
    grid_location_ee = geemap.geopandas_to_ee(geodataframe)
    
else:
    print('Invalid region of interest. Check STATE, COUNTRY, HUC')

In [8]:
##########################################################################################
# Get Sampling Points
##########################################################################################

##########################################################################################
# AKVEG test 04
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'VEG_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
akv_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(akv_geom))
akv_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# ABR_RS test 04
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'ABR_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
abr_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(abr_geom))
abr_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# AKAVA test 04
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'AVA_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
ava_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(ava_geom))
ava_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# NEON
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'NEO_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
neo_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(neo_geom))
neo_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# Seward test 04
di = '/mnt/poseidon/remotesensing/arctic/data/training/Test_05/fcover/'
fi = 'SP_fcover_parent.csv'
obs_data = pd.read_csv(di + fi)

# extract geometry and unique ID
nge_geom = obs_data[['latitude', 
                     'longitude', 
                     'Site Code']]
print(len(nge_geom))
nge_geom.columns = ['latitude', 'longitude', 'Site Code']

##########################################################################################
# combine
obs_geom = pd.concat([akv_geom, abr_geom, ava_geom, neo_geom, nge_geom], 
                     axis=0, 
                     ignore_index=True)
print(len(obs_geom))

# create ee object (feature collection)
obs_geom = obs_geom.reset_index()
obs_points = geemap.df_to_ee(obs_geom,
                             latitude='latitude',
                             longitude='longitude')
print(obs_points.size().getInfo())

##########################################################################################
#sub-select points and extract geometry

# select points that intercept HUC
samplepoints = obs_points.filterBounds(grid_location_ee)

# create dictionary of grid coordinates
points_dict = samplepoints.getInfo()
feats = points_dict['features']

# get ID column
unique_ids = []
for f in feats:
    id = f['properties'][IDCOL]
    unique_ids.append(id)

# Create a list of several ee.Geometry.Polygons
points = []
for f in feats:
    coords = f['geometry']['coordinates']
    point = ee.Geometry.Point(coords)
    # create buffer around point for later reduce regions
    buffered = point.buffer(POINTBUFFER)
    points.append(buffered)

# Make a feature collection for export purposes
points_ee = ee.FeatureCollection(points)
print(f'{len(points)} {POINTBUFFER}-meter buffered points.')

185
107
90
47
86
515
515
514 30-meter buffered points.


In [53]:
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)


img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

m = geemap.Map()
m.set_center(5.2013, 47.3277, 12)
m.addLayer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.addLayer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m

Map(center=[47.3277, 5.2013], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [81]:
start = '2019-06-01'
end = '2019-08-31'

col = (ee.ImageCollection('COPERNICUS/S1_GRD')
       .filterBounds(samplepoints)
       .filterMetadata('instrumentMode', 'equals', 'IW')
       .filterDate(start, end)
       .map(mask_edge))

m = geemap.Map()
m.center_object(samplepoints, zoom=8)
m.addLayer(col.first().select('VV'), {'min': -25, 'max': 5}, 'VV', False)
m.addLayer(col.first().select('VH'), {'min': -25, 'max': 5}, 'VH', False)
m

Map(center=[69.29617727681926, -157.39159744032227], controls=(WidgetControl(options=['position', 'transparent…

In [55]:
passdirection = col.aggregate_array('orbitProperties_pass').getInfo()

In [56]:
print(len([d for d in passdirection if d == 'DESCENDING']))

118


In [57]:
print(len([d for d in passdirection if d == 'ASCENDING']))

14


In [58]:
col = (col.filterMetadata('orbitProperties_pass', 
                          'equals',
                          'DESCENDING'))
# could normalize instead of selecting one ascending/descending

In [59]:
recent = col.median()
proportion = (recent.select('VV')
              .subtract(recent.select('VH')).rename('proportion'))
recent = recent.addBands(proportion)

In [79]:
# RGB is VV, VH, VV-VH
rgb = ee.Image.rgb(recent.select('VV'),
                   recent.select('VH'),
                   recent.select('VV').subtract(recent.select('VH')))
m.addLayer(rgb, {'min':[-18, -25, 1],
                 'max':[0, -10, 0.5]},
           'RGB')

In [ ]:
# monitoring forest change over varying topography can create extra noise
# apply terrain correction
